In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
cd '/content/drive/MyDrive/UBS-Auto_Server_Audit /Detectron'

/content/drive/.shortcut-targets-by-id/1xuo4FVROJBpOYQfz9QcDshp5kPAvpxB1/UBS-Auto_Server_Audit /Detectron


In [15]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.7 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab
# install detectron2:

# !pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html




!pip install -U torch==1.7 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!pip install -U 'git+https://github.com/facebookresearch/detectron2.git@543fd075e146261c2e2b0770c9b537314bdae572'
!pip install -U 'git+https://github.com/facebookresearch/fvcore@4525b814c8bb0f70510e37e68247c958010eb285'





Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html
  Using cached https://download.pytorch.org/whl/cu101/torch-1.7.0%2Bcu101-cp37-cp37m-linux_x86_64.whl (735.3 MB)
  Using cached https://download.pytorch.org/whl/cu101/torchvision-0.6.0%2Bcu101-cp37-cp37m-linux_x86_64.whl (6.6 MB)
  Using cached dataclasses-0.6-py3-none-any.whl (14 kB)
  Using cached torchvision-0.6.0-cp37-cp37m-manylinux1_x86_64.whl (6.6 MB)
INFO: pip is looking at multiple versions of <Python from Requires-Python> to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
  Using cached torch-1.7.0-cp37-cp37m-manylinux1_x86_64.whl (776.7 MB)
ERROR: Cannot install torch==1.7, torchvision==0.6.0 and torchvision==0.6.0+cu101 because these package versions have conflicting dependencies.

The conflict is caused by:
    The user 

1.10.0+cu111 True
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html
  Using cached https://download.pytorch.org/whl/cu101/torch-1.7.0%2Bcu101-cp37-cp37m-linux_x86_64.whl (735.3 MB)
  Using cached https://download.pytorch.org/whl/cu101/torchvision-0.6.0%2Bcu101-cp37-cp37m-linux_x86_64.whl (6.6 MB)
  Using cached dataclasses-0.6-py3-none-any.whl (14 kB)
  Using cached torchvision-0.6.0-cp37-cp37m-manylinux1_x86_64.whl (6.6 MB)
INFO: pip is looking at multiple versions of <Python from Requires-Python> to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a whi

  Cloning https://github.com/facebookresearch/fvcore (to revision 4525b814c8bb0f70510e37e68247c958010eb285) to /tmp/pip-req-build-4fpgiwb5
  Running command git clone -q https://github.com/facebookresearch/fvcore /tmp/pip-req-build-4fpgiwb5
  Running command git rev-parse -q --verify 'sha^4525b814c8bb0f70510e37e68247c958010eb285'
  Running command git fetch -q https://github.com/facebookresearch/fvcore 4525b814c8bb0f70510e37e68247c958010eb285
  Running command git checkout -q 4525b814c8bb0f70510e37e68247c958010eb285


In [16]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
#from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode

In [17]:
def cv2_imshow(im):
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    plt.figure(), plt.imshow(im), plt.axis('off');

In [19]:
import cv2
import os
class_name = ['Printer','Computer monitor','Computer keyboard']
def extract_boxes(img_path,ann_path):
    boxes = list()
    with open(ann_path) as f:
        data_list = f.read().split('\n')
        data_list = [d.split() for d in data_list if d]
        for data in data_list:
            name = ' '.join(data[:-4])
            xmin = int(float(data[-4]))
            ymin = int(float(data[-3]))
            xmax = int(float(data[-2]))
            ymax = int(float(data[-1]))
            coors = [xmin, ymin, xmax, ymax, name]
            if name in class_name:
                boxes.append(coors)
        # directory path
        img = cv2.imread(img_path)
        height, width, _ = img.shape
        return boxes, width, height

def get_balloon_dicts(images_dir,annotations_dir):
    dataset_dicts = []
    for filename in os.listdir(images_dir):
      idx = 0
      if filename[-4:] == '.jpg':
        image_id = filename[:-4]
        img_path = images_dir + filename
        ann_path = annotations_dir + image_id + '.txt'
        boxes, width, height = extract_boxes(img_path,ann_path)
        record = {}
        record["file_name"] = img_path
        record["image_id"] = idx
        record["height"] = height
        record["width"] = width
        objs = []
        for box in boxes:
          obj = {
                  "bbox": box[:-1],
                  "bbox_mode": BoxMode.XYXY_ABS,
                  "category_id": class_name.index(box[-1]), #c[-1]
              }
          objs.append(obj)
        record["annotations"] = objs
        idx = idx + 1
        dataset_dicts.append(record)
    return dataset_dicts


#train
train_images_dir = '/content/drive/MyDrive/mask_rcnn_data/pmk images/printer_img_train/'
train_annotations_dir = '/content/drive/MyDrive/mask_rcnn_data/pmk images/printer_annot_train/'
val_images_dir = '/content/drive/MyDrive/mask_rcnn_data/pmk images/printer_img_test/'
val_annotations_dir = '/content/drive/MyDrive/mask_rcnn_data/pmk images/printer_annot_test/'

DatasetCatalog.register('train', lambda: get_balloon_dicts(train_images_dir,train_annotations_dir))
MetadataCatalog.get('train').thing_classes = class_name

DatasetCatalog.register('val', lambda: get_balloon_dicts(val_images_dir,val_annotations_dir))
MetadataCatalog.get('val').thing_classes = class_name

balloon_metadata = MetadataCatalog.get("train")


AssertionError: ignored

In [ ]:
# To verify the data loading is correct, let's visualize the annotations of randomly selected samples in the training set:

import matplotlib.pyplot as plt
dataset_dicts = get_balloon_dicts(train_images_dir,train_annotations_dir)
for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=balloon_metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    cv2_imshow(out.get_image()[:, :, ::-1])

# **Train Detectron with Retinanet**





In [8]:
cfg = get_cfg()
cfg.OUTPUT_DIR = '/content/drive/MyDrive/UBS-Auto_Server_Audit /Detectron/output_retinanet'
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/retinanet_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/retinanet_R_50_FPN_3x.yaml")  
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00050  
cfg.SOLVER.MAX_ITER = 50  
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3  


os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

Loading config /usr/local/lib/python3.7/dist-packages/detectron2/model_zoo/configs/COCO-Detection/../Base-RetinaNet.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.


[03/29 03:17:10 d2.engine.defaults]: Model:
RetinaNet(
  (backbone): FPN(
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelP6P7(
      (p6): Conv2d(2048, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (p7): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res2)

The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std


[03/29 03:17:21 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.7/dist-packages/detectron2/structures/boxes.py:158: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  tensor = torch.as_tensor(tensor, dtype=torch.float32, device=device)
/usr/local/lib/python3.7/dist-packages/detectron2/structures/boxes.py:158: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  tensor = torch.as_tensor(tensor, dtype=torch.float32, device=device)
/usr/local/lib/python3.7/dist-packages/detectron2/structures/image_list.py:101: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It curre

[03/29 03:18:06 d2.utils.events]:  eta: 0:01:08  iter: 19  total_loss: 1.476  loss_cls: 1.31  loss_box_reg: 0.1719  time: 2.2398  data_time: 0.0240  lr: 0.00019031  max_mem: 3668M
[03/29 03:18:39 d2.utils.events]:  eta: 0:00:18  iter: 39  total_loss: 0.7391  loss_cls: 0.5182  loss_box_reg: 0.1732  time: 1.9390  data_time: 0.0095  lr: 0.00039011  max_mem: 3668M
[03/29 03:18:55 d2.utils.events]:  eta: 0:00:00  iter: 49  total_loss: 0.6275  loss_cls: 0.4578  loss_box_reg: 0.1735  time: 1.8574  data_time: 0.0077  lr: 0.00049001  max_mem: 3854M
[03/29 03:18:55 d2.engine.hooks]: Overall training speed: 48 iterations in 0:01:29 (1.8575 s / it)
[03/29 03:18:55 d2.engine.hooks]: Total training time: 0:01:29 (0:00:00 on hooks)


In [ ]:

cfg.OUTPUT_DIR = '/content/drive/MyDrive/UBS-Auto_Server_Audit /Detectron/output_retinanet'
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8
cfg.DATASETS.TEST = ("val", )
predictor = DefaultPredictor(cfg)

#test evaluation
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

evaluator = COCOEvaluator("val", cfg, False, output_dir='/content/drive/MyDrive/UBS-Auto_Server_Audit /Detectron/output_retinanet')
val_loader = build_detection_test_loader(cfg, "val")
res_val = inference_on_dataset(trainer.model, val_loader, evaluator)

train_evaluator = COCOEvaluator("train", cfg, False, output_dir='/content/drive/MyDrive/UBS-Auto_Server_Audit /Detectron/output_retinanet')
train_loader = build_detection_test_loader(cfg, "train")
res_train = inference_on_dataset(trainer.model, train_loader, train_evaluator)


In [ ]:
res_val

In [11]:
res_train

OrderedDict([('bbox',
              {'AP': 28.33352220669747,
               'AP-Computer keyboard': 29.035362608209798,
               'AP-Computer monitor': 23.132065613557234,
               'AP-Printer': 32.83313839832539,
               'AP50': 49.40383388805384,
               'AP75': 27.825311648686128,
               'APl': 31.310882400043788,
               'APm': 0.0,
               'APs': 0.0})])

In [12]:
from detectron2.utils.visualizer import ColorMode
import matplotlib.pyplot as plt

#Use the final weights generated after successful training for inference  
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8  # set the testing threshold for this model
#Pass the validation dataset
cfg.DATASETS.TEST = ("val", )

predictor = DefaultPredictor(cfg)


base_path = '/content/drive/MyDrive/testing/test_images/pmk_personal_testimages/'
for i in os.listdir(base_path):
  if i[-3:] == 'jpg' or 'jpeg':
    img_path = base_path + i 
    im = cv2.imread(img_path)
    outputs = predictor(im)
    # print(outputs)
    v = Visualizer(im[:, :, ::-1],
                   metadata=balloon_metadata, 
                   scale=0.8,
                   instance_mode=ColorMode.IMAGE   
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu")) #Passing the predictions to CPU from the GPU
    cv2_imshow(v.get_image()[:, :, ::-1])

/usr/local/lib/python3.7/dist-packages/detectron2/structures/image_list.py:101: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride
/usr/local/lib/python3.7/dist-packages/detectron2/modeling/meta_arch/retinanet.py:464: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  anchor_idxs = topk_id

IndexError: ignored